### Model Equations

Let:

- $P(z, t)$: phytoplankton concentration  
- $N(z, t)$: nutrient concentration  
- $\mu$: maximum phytoplankton growth rate  
- $\text{light}(z)$: light intensity at depth  
- $\text{mm}(N) = \frac{N}{N + N_{\text{half}}}$: Michaelis-Menten uptake  
- $d_p$: phytoplankton mortality rate  
- $\lambda$: nutrient relaxation rate  
- $N_0$: background deep nutrient concentration  
- $\kappa(z)$: vertical diffusivity  
- $z$: vertical coordinate (positive upward, so $z < -\text{nutricline depth}$ is deep)  
- $D[\cdot] = \frac{\partial}{\partial z}\left( \kappa(z) \frac{\partial}{\partial z}[\cdot] \right)$: vertical diffusion operator

#### Phytoplankton Equation  
$\frac{\partial P}{\partial t} = \mu \cdot \text{light}(z) \cdot P \cdot \frac{N}{N + N_{\text{half}}} - d_p \cdot P + D[P]$

#### Nutrient Equation 
$\frac{\partial N}{\partial t} = -\mu \cdot \text{light}(z) \cdot P \cdot \frac{N}{N + N_{\text{half}}} + \lambda (N_0 - N) \cdot H(-z - z_n) + D[N]$

where:

- $H(\cdot)$ is the Heaviside step function, equal to 1 when $z < -z_n$ (i.e., below the nutricline depth), and 0 otherwise.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def mm(v, vhalf):
    return v / (v + vhalf)

def ddzkddz(v, kappa, dz):
    nvar = v.shape[0]
    dv = kappa * (v[:,1:] - v[:,:-1]) / dz
    dv = np.hstack((np.zeros((nvar,1)), dv, np.zeros((nvar,1))))
    dv = (dv[:,1:] - dv[:, :-1]) / dz
    return dv

def phytoplankton_model(
    hlight=20,
    kappa_0 = 0.1,
    mixing_sharpness=10,
    nutricline_depth=100,
    dz=1,
    NZ = 150,
    dt=1/16,
    tmax=2000,
    tsnap=10,
    tsnap2=100,
    mu=1.0,
    Nhalf=0.1,
    d_p=0.01,
    lambda_=0.1,
    P0=0.1,
    N0=3.0,
):
    # define light profile
    z = -dz * np.arange(0.5, NZ)
    light = np.exp(z / hlight)

    # define density and mixing profile
    rho0 = 1024  # reference density
    delta_rho = 5.0
    
    rho = rho0 + 0.5 * delta_rho * (1 - np.tanh((z + nutricline_depth) / mixing_sharpness))

    # calculate N^2 (Brunt-Väisälä frequency)
    g = 9.81
    N2 = -g / rho0 * np.gradient(rho, z)

    kappa = kappa_0 * (N2 + 0.01)  # add small value to avoid divide-by-zero
    kappa = 0.5 * (kappa[:-1] + kappa[1:])
    kappa = np.clip(kappa, 1e-5, 1.0) 

    # define nutrient profile scaled by density
    N = N0 * ((rho - rho.min()) / (rho.max() - rho.min()))

    # define tendency function for phytoplankton and nutrient
    def ddt(v, t):
        P = v[0]
        N = v[1]
        uptake = mu * light * P * mm(N, Nhalf)
        dP = uptake - d_p * P
        dN = -uptake + lambda_ * (N0 - N) * (z < -nutricline_depth)
        return np.array([dP, dN])

    # initialize variables
    P = P0 + 0 * z
    v = np.array([P, N])
    t = 0
    snapshots = []
    times = []
    vs = []

    # calculate tendencies and update variables
    # uses a simple Euler method with a half-step correction
    while t < tmax:
        dv = ddt(v, t) + ddzkddz(v, kappa, dz)
        v += dt * dv
        t += dt
        dv = ddt(v, t) + ddzkddz(v, kappa, dz) - dv
        v += (dt / 2) * dv

        if np.remainder(t, tsnap) == 0:
            vs.append(np.max(v, axis=1))
        if np.remainder(t, tsnap2) == 0:
            snapshots.append(v.copy())
            times.append(t)

    return snapshots, times, np.array(vs), z, kappa, light, N0, rho, N2


In [ ]:
NZ = 150

def run_and_plot(hlight=20, kappa_0=0.1, mixing_sharpness=10, nutricline_depth=100):
    snapshots, times, vs, zs, kappa, light, N0, rho, N2 = phytoplankton_model(
        hlight=hlight,
        kappa_0=kappa_0,
        mixing_sharpness=mixing_sharpness,
        nutricline_depth=nutricline_depth,
        NZ=150
    )

    fig, axs = plt.subplots(1, 3, figsize=(8, 5), sharey=True,
                            gridspec_kw={'width_ratios': [2, 1, 1]})

    axs[0].plot(snapshots[-1][0], zs, label='Phytoplankton', color='green')
    axs[0].plot(snapshots[-1][1], zs, label='Nutrient', color='blue')
    axs[0].set_xlabel('Concentration')
    axs[0].set_ylabel('Depth (m)')
    axs[0].legend()

    axs[1].plot(kappa, zs[1:], label='Mixing Coefficient', linestyle='--', color='gray')
    axs[1].set_xlabel(r'$\kappa$ (Mixing Coefficient)')

    axs[2].plot(light, zs, label='Light', linestyle=':', color='orange')
    axs[2].set_xlabel('Light')

    plt.tight_layout()
    plt.show()

# interactive controls for the model parameters
from ipywidgets import interact, FloatSlider, Layout

slider_style = {'description_width': '150px'}
slider_layout = Layout(width='500px') 

interact(run_and_plot,
         hlight=FloatSlider(min=5, max=50, step=1, value=15,
                            description='Attenuation Depth (m)',
                            style=slider_style, layout=slider_layout),
         kappa_0=FloatSlider(min=0, max=10.0, step=0.5, value=1.0,
                             description='κ₀ (m²/s)',
                             style=slider_style, layout=slider_layout),
         mixing_sharpness=FloatSlider(min=1, max=20, step=1, value=10,
                                      description='Mixing Sharpness',
                                      style=slider_style, layout=slider_layout),
         nutricline_depth=FloatSlider(min=5, max=NZ, step=5, value=100,
                                      description='Nutricline Depth (m)',
                                      style=slider_style, layout=slider_layout)
)

interactive(children=(FloatSlider(value=15.0, description='Attenuation Depth (m)', layout=Layout(width='500px'…

<function __main__.run_and_plot(hlight=20, kappa_0=0.1, mixing_sharpness=10, nutricline_depth=100)>